In [1]:
! pip install -Uqq huggingface datasets pandas
! pip freeze

aiohttp==3.9.5
aiosignal==1.3.1
asttokens==2.4.1
async-timeout==4.0.3
attrs==23.2.0
certifi==2024.7.4
charset-normalizer==3.3.2
comm==0.2.2
datasets==2.20.0
debugpy==1.8.2
decorator==5.1.1
dill==0.3.8
exceptiongroup==1.2.2
executing==2.0.1
filelock==3.15.4
frozenlist==1.4.1
fsspec==2024.5.0
huggingface==0.0.1
huggingface-hub==0.24.2
idna==3.7
ipykernel==6.29.5
ipython==8.26.0
jedi==0.19.1
Jinja2==3.1.4
jupyter_client==8.6.2
jupyter_core==5.7.2
MarkupSafe==2.1.5
matplotlib-inline==0.1.7
multidict==6.0.5
multiprocess==0.70.16
nest-asyncio==1.6.0
numpy==2.0.1
packaging==24.1
pandas==2.2.2
parso==0.8.4
pexpect==4.9.0
platformdirs==4.2.2
prompt_toolkit==3.0.47
psutil==6.0.0
ptyprocess==0.7.0
pure_eval==0.2.3
pyarrow==17.0.0
pyarrow-hotfix==0.6
Pygments==2.18.0
python-dateutil==2.9.0.post0
pytz==2024.1
PyYAML==6.0.1
pyzmq==26.0.3
requests==2.32.3
six==1.16.0
stack-data==0.6.3
tornado==6.4.1
tqdm==4.66.4
traitlets==5.14.3
typing_extensions==4.12.2
tzdata==2024.1
urllib3==2.2.2
wcwidth==0.2.13

# PLAN
1. CORPUS: Get an Arabic corpus and split it into sentences.
1. PROCESS: Split into sentences, and divide into train and test.
1. REWORD: Get translation LLMs and translate each sentence back and forth.
1. PARAPHRASE: [OPTIONAL] To generate more sntences, fine-tune a paraphraser on how to link the sentence and its rewording: https://www.youtube.com/watch?v=U6x8_BP69DM
1. SYNTH: Apply morphological analysis to the original and reworded.
1. DIFF: Fine-tune an LLM to tell give a 0 score when asked about the similarity between the synthed original vs the synthed reword.

## 1. CORPUS

In [5]:
import pandas as pd

df = pd.DataFrame([
  {
    'URL': 'https://huggingface.co/datasets/Ali-C137/Hindawi-Books-dataset',
    'size': '46k paragraphs',
    'dialect': 'Literary MSA',
    'sources': 'Hindawi',
    'notes': 'YES. Flowery language.',
  },
  {
    'URL': 'https://huggingface.co/datasets/Tiger14n/arabic_diacritic',
    'size': '600k sentences',
    'dialect': 'Fiqh Arabic',
    'sources': 'Shamela',
    'notes': 'YES. Good grammer.',
  },
  {
    'URL': 'https://huggingface.co/datasets/PRAli22/Arabic_dialects_to_MSA',
    'size': '146k sentences',
    'dialect': 'Dialect <=> MSA',
    'sources': 'Everyday speech?',
    'notes': 'NO. Feels generated.',
  },
  {
    'URL': 'https://huggingface.co/datasets/arbml/Ashaar_diacritized',
    'size': '24k sentences',
    'dialect': 'Dialect & MSA',
    'sources': 'Wikisource Poetry',
    'notes': "NO. I don't want dialects.",
  },
])
df.insert(loc=0, column='name', value=df['URL'].str.replace('https://huggingface.co/datasets/', ''))

df = df.style.format({'URL': lambda val: f'<a target="_blank" href="{val}">LINK</a>'})
df

,name,URL,size,dialect,sources,notes
0,Ali-C137/Hindawi-Books-dataset,LINK,46k paragraphs,Literary MSA,Hindawi,YES. Flowery language.
1,Tiger14n/arabic_diacritic,LINK,600k sentences,Fiqh Arabic,Shamela,YES. Good grammer.
2,PRAli22/Arabic_dialects_to_MSA,LINK,146k sentences,Dialect <=> MSA,Everyday speech?,NO. Feels generated.
3,arbml/Ashaar_diacritized,LINK,24k sentences,Dialect & MSA,Wikisource Poetry,NO. I don't want dialects.


In [6]:
from datasets import load_dataset_builder

ds_builder = load_dataset_builder("Ali-C137/Hindawi-Books-dataset")
print(f"{ds_builder.info.dataset_name}/{ds_builder.info.config_name}")
print(ds_builder.info.splits.keys())
print(ds_builder.info.features.keys())

hindawi-books-dataset/default
dict_keys(['train'])
dict_keys(['BookLink', 'BookName', 'AuthorName', 'AboutBook', 'ChapterLink', 'ChapterName', 'ChapterText', 'AboutAuthor'])


In [7]:
from datasets import load_dataset
dataset = load_dataset("Ali-C137/Hindawi-Books-dataset")
dataset

Generating train split: 100%|██████████| 49821/49821 [00:03<00:00, 15070.96 examples/s]


DatasetDict({
    train: Dataset({
        features: ['BookLink', 'BookName', 'AuthorName', 'AboutBook', 'ChapterLink', 'ChapterName', 'ChapterText', 'AboutAuthor'],
        num_rows: 49821
    })
})

# 2. PROCESS

In [8]:
dataset = dataset.filter(lambda ex: ex['ChapterText'] is not None and type(ex['ChapterText']) is str and len(ex['ChapterText'].strip()) > 0)
dataset

Filter: 100%|██████████| 49821/49821 [00:03<00:00, 13995.38 examples/s]


DatasetDict({
    train: Dataset({
        features: ['BookLink', 'BookName', 'AuthorName', 'AboutBook', 'ChapterLink', 'ChapterName', 'ChapterText', 'AboutAuthor'],
        num_rows: 46274
    })
})

In [9]:
text_distinct_chars = set()
dataset.map(lambda ex: text_distinct_chars.update(ex['ChapterText']))
text_find_distinct_chars = ''.join(sorted(list(text_distinct_chars)))
text_find_distinct_chars

Map: 100%|██████████| 46274/46274 [00:21<00:00, 2126.81 examples/s]


'\t\n !"#$%&\'()*+,-./012345679:;<=>?ABCDEFGHIKLMNOPRSTUWY[\\]_`abcdefghijklmnoprstuvwxyz{|}\xa0£§©«¬®°±µº»×ê÷øˆ˚،؛؟ءآأؤإئابةتثجحخدذرزسشصضطظعغـفقكلمنهوىيًٌٍَُِّْٕٓ٠١٢٣٤٥٦٧٨٩٪٫٬ٰٱٻپچژڠڤڧڨکڬڮگڸڽھۀہۃی۔ۖۗۘۙۚۛۡۤ۩۰۱۲۳۷۸۹\u2002\u2003\u2009\u200a\u200c\u200d\u200e\u200f–—‘’‚“†•…\u202b\u202c\u206c←↑↓−─ﭐﭕﭗﭘﭙﭫﭬﭭﭼﮐﮑﮓﮔﮕﮥﮬﯾﯿﱠﱡ﴾﴿ﷲﷺﺂﺄﺈﺋﺌﺎﺐﺑﺒﺔﺖﺗﺘﺚﺛﺜﺞﺟﺠﺣﺤﺧﺨﺪﺮﺰﺲﺳﺴﺷﺸﺻﺼﺿﻀﻄﻋﻌﻏﻒﻓﻔﻖﻗﻘﻚﻛﻜﻞﻟﻠﻢﻣﻤﻦﻧﻨﻪﻫﻬﻮﻰﻲﻳﻴﻷﻸﻹﻺﻻﻼ'

In [12]:
import re

def process_chapter(text):
  # TOOD improve punctuation handling
  text = re.sub("[\\n\\t ]+", " ", text)
  print(text)
  for i, x in enumerate(re.findall(re.compile(r'(.*?[\.\?…])'), text)):
    if i == 0: continue
    print(x)
  # return text

process_chapter(dataset['train'][0]['ChapterText'])

أَفلَت مطعم ستسوغوان من أية أضرار ناتجة عن الحرب؛ لأنه لحُسن الحظ يقع على تلة عالية في منطقة كوايشيكاوا كثيفة التعرُّجات بطوكيو. فلم تُصَب بأذى الحديقة الشهيرة التي على نسق مدرسة كوبوريئنشو ذات السمعة العريقة التي تصل مساحتها إلى عشرة آلاف متر مربع، ولا أيضًا بوابة تشوجاكو التي نُقِلَت من أحد معابد كيوتو الشهيرة، ولا المدخل وحجرة الضيوف اللتان نُقِلَتا كما هما من معبد بوذي قديم في مدينة نارا، ولا القاعة الكبرى التي أُنشِئت فيما بعد. في خضمِّ الجلبة التي حدثت بسبب ضريبة الأملاك بعد الحرب، انتقل مبنى ستسوغوان من يد مالكه الأصلي فنان صناعة الشاي ورجل الأعمال الشهير، إلى يد مالكةٍ جميلةٍ وذات نشاط وحيوية، فتحوَّل على الفور إلى مطعم ذي شهرة عريضة. تلك المالكة اسمها كازو فوكوزاوا. كانت كازو ذات براءة ريفية مع مظهر رائع الجمال، وكانت تفيض قوةً وحماسًا على الدوام. يخجل الشخص الذي يَتحرَّك قلبه بهمَّة وحماس من تعقيدات نفسه، عندما يرى كازو، وعلى الأرجح يَتشجَّع الشخص الذي ارتخت قواه المعنوية عندما يرى كازو، أو على العكس يحس بشعور الانسحاق التام. تستطيع تلك المرأة التي جمعت في جسد واحد من خلال نع